# Watson OpenScale Mortgage Default Lab - Model Feed

Part two of a hands-on lab for IBM Watson OpenScale, this notebook should be run in a Watson Studio project with Python 3.6 or greater. This notebook will use the Watson Machine Learning client to send scoring requests to the model. You can specify the number of requests sent in the cells below.

Replace the credentials below with those of your Watson Machine Learning instance with the mortgage default model deployed.

In [1]:
WML_CREDENTIALS = {
  "apikey": "ClPV2HAgLhmtNxM7fSpJoVcD-4dEDnHqzdVgAK8uqWgv",
  "iam_apikey_description": "Auto-generated for key d0a06315-c1c1-4e1d-9431-eefdb8b06026",
  "iam_apikey_name": "Service credentials-cpat-f2f-wml",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/82cf08a3d55d4f8fa8266f348d7f4362::serviceid:ServiceId-092953e7-cef8-405a-a3a5-123d5107186d",
  "instance_id": "e88e90be-acaa-4350-954a-6d27a9e785bd",
  "url": "https://us-south.ml.cloud.ibm.com"
}

In the cell below, ensure that the model and deployment names are the same used for creating the model, and set the minimum and maximum number of records you wish to send to the model for scoring.

In [2]:
MODEL_NAME = 'Mortgage Default'
DEPLOYMENT_NAME = 'Mortgage Default - Production'

MIN_PAYLOADS = 101
MAX_PAYLOADS = 125

Using the credentials from above, create a Watson Machine Learning API client.

In [3]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

client = WatsonMachineLearningAPIClient(WML_CREDENTIALS)

Get the deployment and model IDs from the WML client.

In [4]:
mortgage_deployment_id = None
model_deployment_ids = client.deployments.get_uids()
for deployment_id in model_deployment_ids:
    deployment = client.deployments.get_details(deployment_id)
    model_id = deployment['entity']['deployable_asset']['guid']
    if deployment['entity']['name'] == DEPLOYMENT_NAME:
        print('Deployment id:', deployment_id)
        mortgage_deployment_id = deployment_id
        print('Model id:', model_id)
        break

Deployment id: 351cdae8-5faf-472b-a90c-385f0623ce2b
Model id: 6f956465-f430-4a33-a723-3e206be21809


Get the scoring URL of the model.

In [5]:
deployment_details = client.deployments.get_details(mortgage_deployment_id)
scoring_url = deployment_details['entity']['scoring_url']
scoring_url

'https://us-south.ml.cloud.ibm.com/v3/wml_instances/e88e90be-acaa-4350-954a-6d27a9e785bd/deployments/351cdae8-5faf-472b-a90c-385f0623ce2b/online'

Download the JSON records from github.

In [6]:
!rm mortgage_feed.json
!wget https://raw.githubusercontent.com/emartensibm/mortgage-default/master/mortgage_feed.json

rm: cannot remove ‘mortgage_feed.json’: No such file or directory
--2020-03-17 02:01:51--  https://raw.githubusercontent.com/emartensibm/mortgage-default/master/mortgage_feed.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.36.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.36.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 32112 (31K) [text/plain]
Saving to: ‘mortgage_feed.json’

100%[======================================>] 32,112      --.-K/s   in 0.1s    

2020-03-17 02:01:52 (282 KB/s) - ‘mortgage_feed.json’ saved [32112/32112]



In [7]:
import json

with open('mortgage_feed.json', 'r') as scoring_file:
    data = json.load(scoring_file)

Get the feature fields from the data file, dropping the transaction ID.

In [8]:
data['fields'][1:]

['Income',
 'AppliedOnline',
 'Residence',
 'Yrs_at_Current_Address',
 'Yrs_with_Current_Employer',
 'Number_of_Cards',
 'Creditcard_Debt',
 'Loans',
 'Loan_Amount',
 'SalePrice',
 'Location']

Set up the data structure for scoring requests. The ID field was not a feature used to train the model, but is still used to identify the transactions. For this reason, we will send it as metadata with the request, and it will be stored in the OpenScale payload log.

In [9]:
scoring_payload = {
    "fields": data['fields'][1:],
    "values": [],
    "meta":{
        "fields": ["ID"],
        "values": []
    }
}

Randomly select fields from the data to score, and randomly assign an integer ID to each.

In [10]:
import random
import string

letters = string.digits

for _ in range(0, random.randint(MIN_PAYLOADS, MAX_PAYLOADS)):
    value_to_score = random.choice(data['values'])
    scoring_payload['values'].append(value_to_score[1:])
    scoring_payload['meta']['values'].append([int(''.join(random.choices(letters, k=8)))])

Send the scoring request to the model.

In [11]:
predictions = client.deployments.score(scoring_url, scoring_payload)

Print out the returned predictions.

In [12]:
predictions

{'fields': ['prediction', 'probability'],
 'values': [['YES', [0.0, 1.0]],
  ['YES', [0.0, 1.0]],
  ['YES', [0.0, 1.0]],
  ['YES', [0.4, 0.6]],
  ['YES', [0.0, 1.0]],
  ['NO', [1.0, 0.0]],
  ['YES', [0.1, 0.9]],
  ['NO', [0.8, 0.2]],
  ['YES', [0.3, 0.7]],
  ['YES', [0.0, 1.0]],
  ['YES', [0.0, 1.0]],
  ['YES', [0.0, 1.0]],
  ['NO', [1.0, 0.0]],
  ['YES', [0.0, 1.0]],
  ['NO', [0.8, 0.2]],
  ['NO', [0.6, 0.4]],
  ['YES', [0.1, 0.9]],
  ['YES', [0.0, 1.0]],
  ['NO', [0.9, 0.1]],
  ['NO', [0.9, 0.1]],
  ['YES', [0.0, 1.0]],
  ['NO', [1.0, 0.0]],
  ['YES', [0.1, 0.9]],
  ['YES', [0.0, 1.0]],
  ['NO', [0.8, 0.2]],
  ['NO', [0.9, 0.1]],
  ['NO', [0.5, 0.5]],
  ['YES', [0.0, 1.0]],
  ['YES', [0.3, 0.7]],
  ['YES', [0.1, 0.9]],
  ['NO', [0.8, 0.2]],
  ['NO', [0.7, 0.3]],
  ['YES', [0.0, 1.0]],
  ['YES', [0.14166666666666666, 0.8583333333333334]],
  ['YES', [0.2, 0.8]],
  ['YES', [0.1, 0.9]],
  ['NO', [0.9, 0.1]],
  ['YES', [0.1, 0.9]],
  ['NO', [0.9, 0.1]],
  ['YES', [0.0, 1.0]],
  ['YES', [0

# Congratulations!

If all cells have run successfully, you have submitted data to your model. You can save a version of this notebook using the **File** menu above, and schedule it to run periodically to continue sending data for OpenScale to monitor.

This menu can be scheduled to run regularly to feed data to the model and trigger the OpenScale monitors. To schedule it, click **File**->**Save Version**. Once the notebook has saved, click the **clock icon** to the right of the **share icon** in the top center of the screen and follow the prompts. Running it hourly will provide the best results.